In [1]:
#import library
#install library
!pip install caret
!pip install vroom
from IPython.display import clear_output 



clear_output()

In [2]:
import pandas as pd
import caret
import vroom
import numpy as np
from plotnine import ggplot, aes, geom_line

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bike-sharing-demand/sampleSubmission.csv
/kaggle/input/bike-sharing-demand/train.csv
/kaggle/input/bike-sharing-demand/test.csv


In [4]:
#Import Data
train = pd.read_csv("//kaggle/input/bike-sharing-demand/train.csv")
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")
submission = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")

test['count'] = np.nan

In [5]:
#Convert string to Datetime
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

#Split Datetime
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['DOW'] = train['datetime'].dt.dayofweek
train['hour'] = train['datetime'].dt.hour
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['DOW'] = test['datetime'].dt.dayofweek
test['hour'] = test['datetime'].dt.hour

In [6]:

train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,DOW,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,2011,1,5,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,2011,1,5,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2011,1,5,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,2011,1,5,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,2011,1,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,2012,12,2,19
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,2012,12,2,20
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,2012,12,2,21
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,2012,12,2,22


In [7]:
train.corr()['count']
#Looking at the dataset, it looks like count = casual + registered
# Variables like holiday, weather, atemp (similar to temp), DOW has no impact to count

season        0.163439
holiday      -0.005393
workingday    0.011594
weather      -0.128655
temp          0.394454
atemp         0.389784
humidity     -0.317371
windspeed     0.101369
casual        0.690414
registered    0.970948
count         1.000000
year          0.260403
month         0.166862
DOW          -0.002283
hour          0.400601
Name: count, dtype: float64

In [8]:
col = ['workingday','temp','humidity', 'year','month', 'hour']
x = train[col]
y = train['count']

In [9]:
X_test = test[col]
Y_test = test['count']

In [10]:
#To apply RSMLE function to validate the accurancy
#Evaluation Score function
def RSMLE(predictions , realizations):
    predictions_use = predictions.clip(0)
    rmsle = np.sqrt(np.mean(np.array(np.log(predictions_use+1)-np.log(realizations+1))**2))
    return rmsle

In [11]:
#To test the current model, split the train set
from sklearn.model_selection import train_test_split
X_train , X_valid ,Y_train , Y_valid = train_test_split(x,y,test_size = 0.25, random_state = 201)



###Model 1. XG - Boost
Features used: 'season','workingday','temp','humidity', 'year','month', 'hour'

In [12]:
# try xgboost
import xgboost as xgb
xgb_train = xgb.DMatrix(X_train,label=Y_train)
xgb_valid = xgb.DMatrix(X_valid)

In [13]:
param = { 'max_depth': 6 , 'eta':0.1 , 'seed' : 201 , 'objective' : 'reg:linear'}
xgb.cv(param,xgb_train,500,nfold=5,show_stdv=False,verbose_eval=True,as_pandas=False)

[22:00:14] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:14] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:14] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:14] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:14] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:241.98124	test-rmse:242.05036
[1]	train-rmse:222.11462	test-rmse:222.25468
[2]	train-rmse:204.37448	test-rmse:204.59467
[3]	train-rmse:188.84675	test-rmse:189.20102
[4]	train-rmse:174.60267	test-rmse:175.16819
[5]	train-rmse:162.24759	test-rmse:162.96598
[6]	train-rmse:151.16189	test-rmse:151.93352
[7]	train-rmse:141.34811	test-rmse:142.26565
[8]	train-rmse:132.32517	test-rmse:133.27009


{'train-rmse-mean': [241.9812438,
  222.114621,
  204.3744814,
  188.8467496,
  174.60266719999998,
  162.24758619999997,
  151.16188960000002,
  141.34811100000002,
  132.32517079999997,
  123.87433939999998,
  115.93954019999998,
  108.55969380000002,
  102.4626252,
  96.538002,
  91.3278504,
  86.98254239999999,
  83.37709659999999,
  79.61436619999999,
  76.40553739999999,
  73.7010604,
  71.10502919999999,
  67.965599,
  65.1173828,
  62.3607474,
  60.325528799999994,
  58.240821800000006,
  56.7694886,
  55.651434200000004,
  54.3806328,
  53.34728640000001,
  52.3426598,
  51.6848724,
  50.859750999999996,
  50.1785536,
  49.598045400000004,
  48.935277400000004,
  48.366922599999995,
  47.7038812,
  47.2832206,
  46.9160822,
  46.38303440000001,
  45.957370000000004,
  45.5834786,
  45.1315576,
  44.89933620000001,
  44.613454399999995,
  44.40044400000001,
  44.097819400000006,
  43.8384278,
  43.56651240000001,
  43.321725400000005,
  43.135003600000005,
  42.9386582,
  42.69

In [14]:
num_round = 400
xg_model = xgb.train(param,xgb_train,num_round)
xg_pred = xg_model.predict(xgb_valid)

[22:00:24] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:25] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


In [15]:
xg_model.get_fscore()
#Season plays a fair role in the model


{'hour': 5750,
 'workingday': 1626,
 'month': 2712,
 'year': 1240,
 'temp': 4838,
 'humidity': 5083}

In [16]:

RSMLE(xg_pred,Y_valid)

0.4696547196210395

In [17]:
#Apply the model into whole dataset

xgbtrain = xgb.DMatrix(x,label=y)
xgbtest = xgb.DMatrix(X_test)
xgmodel = xgb.train(param,xgbtrain,400)
xgpred = xgmodel.predict(xgbtest)
xg_clipped = pd.Series(xgpred.clip(0))

[22:00:26] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:28] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


In [18]:
submission['count'] = xg_clipped
submission.to_csv('submission.csv', index=False)